# VGG16 TESTING

In [1]:
import torchvision.models as models
from time import time
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from data import input_data
import numpy as np
from collections import OrderedDict

In [2]:
vgg16 = models.vgg16(pretrained="imagenet")
device = torch.device("cuda:1")

In [3]:
classifier = nn.Sequential(nn.Linear(25088,4096),
                                nn.ReLU6(True),
                                nn.Dropout(0.5,inplace=False),
                                nn.Linear(4096,4096),
                                nn.ReLU6(True),
                                nn.Dropout(0.5,inplace=False),
                                nn.Linear(4096,10))

In [4]:
f = []
for i in [i for i in classifier.state_dict().keys()][:-2]:
    f.append(((i,vgg16.classifier.state_dict()[i]))) 
for i in [i for i in classifier.state_dict().keys()][-2:]:
    f.append(((i,classifier.state_dict()[i]))) 

In [5]:
vgg16.classifier = classifier
vgg16.classifier.load_state_dict(OrderedDict(f),strict=True)
vgg16.to(device)
print('')

In [6]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
optimizer = optim.Adam(vgg16.parameters(), lr=0.0001)
criterion_d = nn.CrossEntropyLoss()
input_train = input_data(root_dir = "/home/hemant/net/easy_net/data/train/", type = "valid")
train_dl =  DataLoader(input_train, batch_size=64,shuffle=True, num_workers=4)
input_valid = input_data(root_dir = "/home/hemant/net/easy_net/data/test/", type = "valid")
valid_dl =  DataLoader(input_valid, batch_size=64,shuffle=False, num_workers=4)

total number of classes in valid are :  10
size of data 9984
total number of classes in valid are :  10
size of data 996


# Training

In [12]:
vgg16.load_state_dict(torch.load("../weights/vgg16.pth"))

<All keys matched successfully>

In [ ]:
stat = 0
for j in range(50):
    print("start of epoch: ", j+1)
    #Training
    running_loss = 0
    start = time()
    vgg16.train()
    for i, data in enumerate(train_dl, 0):

        input, target, img_name, number_of_class = data
        input, target = (input.type(torch.float32)).to(device), target.to(device)

        out = vgg16(input)

        loss = criterion_d(out, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()


        optimizer.zero_grad()
        
        # print every 25 mini-batches
        if i % 25 == 24:
            print('[%d, %5d] loss: %.3f' %(j + 1, i + 1, running_loss))
            running_loss = 0
    end = time()
    print("It took : ", (end - start)/60, " mins for the last training epoch")
    
    running_loss, acc, num, length = 0, 0, 0, 0
    with torch.no_grad():
        start = time()
        for i, data in tqdm(enumerate(valid_dl, 0),total=len(valid_dl), unit="images",position=0,leave=True):
            vgg16.eval()

            input, target, img_name, number_of_class = data
            input, target = (input.type(torch.float32)).to(device), target.to(device)

            out = vgg16(input)

            loss = criterion_d(out, target)
            running_loss += loss.cpu().numpy()
            out , predicted = torch.max(out, 1)
            for k in range(len(target)):
                if target[k] == predicted[k].item():
                    num = num + 1
            length = length + len(target)
        acc = (num/length)*100
        end = time()
        print("accuracy and val loss is : ",acc,",",running_loss/(i+1), " --AND-- ", " It took : ", (end - start), " seconds ")
    
    if acc > stat:
        stat = acc
        torch.save(vgg16.state_dict(),"../weights/" + "vgg16" + ".pth")

start of epoch:  1
[1,    25] loss: 1.913
[1,    50] loss: 1.966
[1,    75] loss: 2.105
[1,   100] loss: 2.289
[1,   125] loss: 2.240
[1,   150] loss: 2.226
It took :  1.1645395835240682  mins for the last training epoch



accuracy and val loss is :  86.94779116465864 , 0.529678113758564  --AND--   It took :  7.521206617355347  seconds 
start of epoch:  2
[2,    25] loss: 2.082
[2,    50] loss: 1.722
[2,    75] loss: 2.677
[2,   100] loss: 2.986
[2,   125] loss: 1.557
[2,   150] loss: 1.955
It took :  1.1477917989095052  mins for the last training epoch



accuracy and val loss is :  83.83534136546184 , 0.7091823238879442  --AND--   It took :  7.784949541091919  seconds 
start of epoch:  3
[3,    25] loss: 2.359
[3,    50] loss: 1.463
[3,    75] loss: 0.827
[3,   100] loss: 0.541
[3,   125] loss: 0.752


# Testing

In [13]:
running_loss, acc, num, length = 0, 0, 0, 0
with torch.no_grad():
    start = time()
    for i, data in tqdm(enumerate(valid_dl, 0),total=len(valid_dl), unit="images",position=0,leave=True):
        vgg16.eval()

        input, target, img_name, number_of_class = data
        input, target = (input.type(torch.float32)).to(device), target.to(device)

        out = vgg16(input)

        loss = criterion_d(out, target)
        running_loss+=loss.cpu().numpy()

        out , predicted = torch.max(out, 1)
        for k in range(len(target)):
            if target[k] == predicted[k].item():
                num = num + 1
        length = length + len(target)
    acc = (num/length)*100
    end = time()
    print("accuracy and val loss is : ",np.round(acc,3),",",np.round(running_loss/(i+1),3), " --AND-- ", " It took : ", (end - start), " seconds ")


accuracy and val loss is :  87.149 , 0.567  --AND--   It took :  8.11099100112915  seconds 


In [ ]:
# torch.save(vgg16.state_dict(),"../weights/" + "vgg16" + ".pth")

In [ ]:
vgg16